Actividad 3.

1. Generar preguntas de interes o hipotesis de interes sobre el dataset elegido para el proyecto final.
2. Crear visualizaciones (univariados, bivariados o trivariados) junto con resumenes numericos basicos acordes con los tipos de variables disponibles.
3. Interpretar los resultados obtenidos.

In [1]:
import pandas as pd

Vacunas = pd.read_csv("Asignacion_vacuna_contra_COVID-19.csv")
Muertes = pd.read_csv("Fallecidos_COVID_en_Colombia.csv")

Preguntas 

1. Cuantas muertes por covid-19 se reportan por region?
2. Cuantas vacunas se aplicaron discriminadas por laboratorio por cada region?
3. Existe relacion en el tipo de vacuna mas aplicada en cada region y cantidad de muertes.

In [ ]:
# Agrupar Muertes por Region (Departamento)
muertes_por_region = Muertes.groupby('Nombre departamento').size().reset_index(name='Muertes')

# Por comodidad vamos a trabajar con el top 10 de regiones con mayor numero de muertes
top_regiones = muertes_por_region.nlargest(10, 'Muertes')


print(top_regiones)

En esta parte se analizan los datos contentidos en el Dataframe Vacunas y se observa que algunas para el mismo labortatorio de vacunas el archivo las reporta con diferentes formas de escritura como como en el caso de Astrazeneca. Mismo escenario para los territorios.

Se normaliza la data y se filtra para tener una tabla con mejor visibilidad y que contenga solo la informacion deseable para el analisis.

Para normalizar la data se crean dos diccionarions que reemplazaran la informacion contenida en el dataset para grupar los laboratorios y los territorios.

In [ ]:
# Normalizacion de las variables

labs_corrections = {
    'ASTRAZENECA': 'AZTRAZENECA',
    'astrezenca': 'AZTRAZENECA',  
    'pfizer': 'PFIZER',          
    'sinovac ': 'SINOVAC'         
}
Vacunas['Laboratorio_Vacuna'] = Vacunas['Laboratorio_Vacuna'].replace(labs_corrections)

territorio_corrections = {
    "ATLÁNTICO" : "ATLANTICO",
    'ATLANTICO' : 'ATLANTICO',
    'SANTA MARTHA' : 'SANTA MARTA',
    'SANTA MARTA' : 'SANTA MARTA',
    'BOGOTÁ D.C.' : 'BOGOTA D.C.',
    'BOGOTA D.C.' : 'BOGOTA D.C.',
    'BOYACÁ' : 'BOYACA',
    'BOYACA' : 'BOYACA',
    'CAQUETÁ' : 'CAQUETA',
    'CAQUETA' : 'CAQUETA',
    'CHOCÓ' : 'CHOCO',
    'CHOCO' : 'CHOCO',
    'CÓRDOBA' : 'CORDOBA',
    'CORDOBA' : 'CORDOBA',
    'LA_GUAJIRA' : 'LA GUAJIRA',
    'LA GUAJIRA' : 'LA GUAJIRA',
    'QUINDÍO' : 'QUINDIO',
    'QUINDIO' : 'QUINDIO',
    'VALLE_DEL_CAUCA' : 'VALLE DEL CAUCA',
    'VALLE DEL CAUCA' : 'VALLE DEL CAUCA',
    'VAUPÉS' :  'VAUPES',
    'VAUPES' :  'VAUPES',
    'SAN ANDRES, PROVIDENCIA Y SANTA CATALINA' :'SAN ANDRES',
    'BOLÍVAR' : 'BOLIVAR',
    'BOLIVAR': 'BOLIVAR'
}

Vacunas['Nom_Territorio'] = Vacunas['Nom_Territorio'].replace(territorio_corrections)

# Filtrado de Datos 

vacunas_filtradas = Vacunas[['Nom_Territorio', 'Laboratorio_Vacuna', 'Cantidad']].dropna()

vacunas_pivot = vacunas_filtradas.pivot_table(
    index='Nom_Territorio',
    columns='Laboratorio_Vacuna',
    values='Cantidad',
    aggfunc='sum',
    fill_value=0
).reset_index()

vacunas_pivot.columns.name = None  

print(vacunas_pivot)

In [ ]:
import plotly.express as px

# Melt the pivoted DataFrame for Plotly
melted_data = vacunas_pivot.melt(
    id_vars='Nom_Territorio', 
    var_name='Laboratorio_Vacuna', 
    value_name='Cantidad'
)

melted_data['Cantidad'] = melted_data['Cantidad'].fillna(0)

# Plot a bar chart
fig = px.bar(
    melted_data,
    x='Laboratorio_Vacuna', 
    y='Cantidad', 
    color='Laboratorio_Vacuna',
    title='Vacunas aplicadas por laboratorio en todas las regiones',
    labels={'Laboratorio_Vacuna': 'Laboratorio', 'Cantidad': 'Cantidad Aplicada'}
)

# Display the chart
fig.show()

De acuerdo con lo anterior, las zonas que reportaron mas muertes por Covid-19 fueron Bogota, Antioqui, Valle del Cauca y Santander. 
En cuanto a las dosis aplicadas, el laboratorio que aplico mas vacunas en todo el territorio nacional fue Pfizer, seguido de Sinovac.

Existe relacion entre vacunas apliacadas y muertes por region?

In [ ]:
vacunas_filtradas.rename(columns={"Nom_Territorio": "Nombre departamento"}, inplace=True)

vacunas_filtradas.columns = vacunas_filtradas.columns.str.strip()
top_regiones.columns = top_regiones.columns.str.strip()

#print("Vacunas columns:", vacunas_filtradas.columns.tolist())
#print("Muertes columns:", top_regiones.columns.tolist())

vacunas_filtradas.rename(columns={"Nombre departamento": "Departamento"}, inplace=True)
top_regiones.rename(columns={"Nombre departamento": "Departamento"}, inplace=True)

df_merged = pd.merge(top_regiones, vacunas_filtradas, on="Departamento")

print(df_merged.info())
print(df_merged.head())

Para el analisis de correlacion se utilizo el meto "merge" para crear una columna en comun en ambos datastes la cual lleva como nombre "Departamento"

In [23]:
correlation_value = df_merged["Muertes"].corr(df_merged["Cantidad"])
print(f"Corelacion entre muertes y cantidad de vacuneas aplicadas: {correlation_value}")

Corelacion entre muertes y cantidad de vacuneas aplicadas: 0.43919942488017405


Una correlacion de 0.43 impliaca que, pese a que hay una correlacion positiva entre la cantidad de muertes por Covid-19 y la cantidad de dosis aplicadas, no es posible determinar que las muertes se deban a la aplicacion de vacunas. Se podria decir que a mayor cantidad de infectados, mayor cantidad de dosis aplicadas y que las muertes se pueden deber a la gravedad del brote de covid-19 y a otros factores externos, como la edad, enfermedades de base, entre otros.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df_merged["Cantidad"], df_merged["Muertes"], alpha=0.5)
plt.xlabel("Vaccines Applied")
plt.ylabel("COVID-19 Deaths")
plt.title("Correlation Between Vaccination and Deaths")
plt.show()